In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/calorie-expenditure/__results__.html
/kaggle/input/calorie-expenditure/cat_oof_preds.csv
/kaggle/input/calorie-expenditure/__notebook__.ipynb
/kaggle/input/calorie-expenditure/cat_test_preds.csv
/kaggle/input/calorie-expenditure/__output__.json
/kaggle/input/calorie-expenditure/custom.css
/kaggle/input/calorie-expenditure/models/cat16.cbm
/kaggle/input/calorie-expenditure/models/cat15.cbm
/kaggle/input/calorie-expenditure/models/cat6.cbm
/kaggle/input/calorie-expenditure/models/cat18.cbm
/kaggle/input/calorie-expenditure/models/cat8.cbm
/kaggle/input/calorie-expenditure/models/cat7.cbm
/kaggle/input/calorie-expenditure/models/cat19.cbm
/kaggle/input/calorie-expenditure/models/cat3.cbm
/kaggle/input/calorie-expenditure/models/cat14.cbm
/kaggle/input/calorie-expenditure/models/cat1.cbm
/kaggle/input/calorie-expenditure/models/cat17.cbm
/kaggle/input/calorie-expenditure/models/cat11.cbm
/kaggle/input/calorie-expenditure/models/cat10.cbm
/kaggle/input/calorie-expenditure/mode

In [2]:
seed = 1
splits=10
np.random.seed(seed)

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')
samp_sub = pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')

In [4]:
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.metrics import mean_squared_log_error
import warnings
def cv_score(X, y, model_dict, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    scores = {name: [] for name in model_dict.keys()}
    
    for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y), 1):
        # Split original data
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train_orig, y_valid_orig = y.iloc[train_idx], y.iloc[valid_idx]

        # 1. Box-Cox Transformation (Per Fold)
        y_train_bc, lambda_fold = stats.boxcox(y_train_orig + 1)  # Add 1 to handle zeros
        y_train_bc = pd.Series(y_train_bc, index=y_train_orig.index)

        # # 2. AutoFeat Engineering (Per Fold)
        # af = AutoFeatRegressor()
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore")
        #     X_train_fe = af.fit_transform(X_train, y_train_bc)
        #     X_valid_fe = af.transform(X_valid)

        # 3. Model Training & Validation
        for name, model in model_dict.items():
            reg = clone(model)
            
            if isinstance(reg, cat.CatBoostRegressor):
                train_pool = Pool(X_train, y_train_bc)
                reg.fit(train_pool, verbose=0)
                preds_bc = reg.predict(X_valid)
            else:
                reg.fit(X_train, y_train_bc)
                preds_bc = reg.predict(X_valid)

            # Inverse transformation
            preds = inv_boxcox(preds_bc, lambda_fold) - 1  # Subtract added constant
            preds = np.clip(preds, 1e-6, None)
            
            # Calculate RMSLE
            rmsle = np.sqrt(mean_squared_log_error(y_valid_orig, preds))
            scores[name].append(rmsle)
            print(f"Fold {fold} | {name} | RMSLE: {rmsle:.5f}")

    # Calculate final scores
    print("\n=== Final Scores ===")
    for name in model_dict:
        avg_score = np.mean(scores[name])
        std_score = np.std(scores[name])
        print(f"{name}: {avg_score:.5f} ± {std_score:.5f}")
    
    return scores

In [5]:
# !pip3 install autofeat

In [6]:
# from autofeat import AutoFeatRegressor
from scipy import stats
from scipy.special import inv_boxcox

sex_mapping = {
    'male':0,
    'female':1
}
train['Sex'] = train['Sex'].map(sex_mapping)
test['Sex'] = test['Sex'].map(sex_mapping)
X = train.drop(columns=['Calories'])
y = train['Calories']
y_bc, lambda_ = stats.boxcox(y)
y_bc = pd.Series(y_bc)


In [7]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from catboost import Pool
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
import shap
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline


params = {
    'learning_rate': 0.013746281229599575,
    'depth': 10,
    'l2_leaf_reg': 2.6989912450312117,
    'random_strength': 2.1007809924141934,
    'grow_policy': 'SymmetricTree',
    'bootstrap_type': 'Bernoulli',
    'iterations': 2249,
    'border_count': 100,
    'has_time': True,
    'subsample': 0.9297145525388769,
}
model = cat.CatBoostRegressor(random_state=seed, verbose=0, task_type='GPU', **params)
# model.fit(X_train_new, y_bc)
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(X_train_new)

# importance_features = pd.DataFrame(
#     {'feature':X_train_new.columns,
#     'importance': np.abs(shap_values).mean(0)}
# ).sort_values('importance', ascending=False)
# top_features = importance_features.head(5)['feature'].tolist()
model_dict = {
    # 'xgb': xgb.XGBRegressor(random_state=seed),
    # 'lgb': lgb.LGBMRegressor(random_state=seed, verbosity=0),
    # 'etr': ExtraTreesRegressor(random_state=seed, n_jobs=-1),
    'cat': cat.CatBoostRegressor(random_state=seed, verbose=0,task_type='GPU', **params),
    # 'hgb': HistGradientBoostingRegressor(random_state=seed)}
}


In [8]:
# X_train_new = pd.read_csv('/kaggle/input/calorie-expenditure/X_train_new_2step.csv')
# X_test_new = pd.read_csv('/kaggle/input/calorie-expenditure/X_test_new_2step.csv')
# cols = ['Sex',
#  'Duration',
#  'Heart_Rate',
#  '1/Duration',
#  'Age*Duration**2',
#  'log(Duration)/Duration',
#  'sqrt(Age)*log(Duration)',
#  'sqrt(Age)*Heart_Rate**3']
# X_train_new = X_train_new[cols]
# X_test_new = X_test_new[cols]

In [9]:
def feature_engineering(df):
    df = df.copy()
    # df['bmr_proxy'] = (
    #     (df['Weight'] * 10) + 
    #     (df['Height'] * 6.25) - 
    #     (df['Age'] * 5) + 
    #     np.where(df['Sex'] == 0, 5, -161)  # Vectorized conditional
    # )
    # # 2. Cardiovascular Efficiency Score
    # df['cardio_eff'] = df['Heart_Rate'] * df['Duration'] / (df['Age']**0.5)

    # # 3. Thermal Metabolic Load
    # df['thermal_load'] = (df['Body_Temp'] - 36.5) * df['Duration']

    # 4. Body Mass Index Squared Interaction
    df['bmi_hr'] = (df['Weight']/(df['Height']/100)**2) * df['Heart_Rate']

    # 5. Age-Weighted Intensity
    # df['age_intensity'] = (220 - df['Age'] - df['Heart_Rate']) * df['Duration']

    # # 6. Metabolic Equivalent (MET) Estimate
    # df['met_estimate'] = (df['Heart_Rate'] * df['Body_Temp']) / (df['Age']**0.3)

    # # 7. Oxygen Consumption Proxy
    # df['vo2_max_proxy'] = (df['Heart_Rate'] * df['Duration']) / (df['Weight']**0.5)

    # # 8. Gender-Specific Metabolic Rate
    # df['sex_metabolic'] = df['Sex'] * (df['bmr_proxy'] / df['Age'])

    # # 9. Body Surface Area (DuBois formula)
    # df['bsa'] = 0.007184 * (df['Height']**0.725) * (df['Weight']**0.425)

    # # 10. Heart Rate Reserve
    # df['hr_reserve'] = (df['Heart_Rate'] / (220 - df['Age'])) * df['Duration']

    # # 11. Thermic Effect of Exercise
    # df['tee'] = np.log1p(df['Duration']) * (df['Body_Temp'] - 36.5) * df['Weight']

    # # 12. Power-to-Weight Ratio
    # df['power_weight'] = (df['Duration'] * df['Heart_Rate']) / df['Weight']

    # # 13. Age-Decay Adjusted Calories
    # df['age_decay'] = np.exp(-df['Age']/50) * df['Duration'] * df['Heart_Rate']

    # # 14. Body Composition Index
    # df['bci'] = (df['Height']/100) / (df['Weight']**0.3) * df['Heart_Rate']

    # # 15. Cumulative Exertion
    # df['cumulative_exertion'] = (df['Heart_Rate'] * df['Body_Temp']) / (
    # df['Weight'] * (df['Age']**0.2))
    return df.copy()

X_new = feature_engineering(X)

In [10]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.base import clone
import warnings
warnings.filterwarnings('ignore')

def generate_oof_predictions(X, y, model_dict, cv_folds=5, seed=42, categorical_features=None):
    n_samples = len(X)
    oof_predictions = pd.DataFrame(index=range(n_samples))
    
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=seed)
    
    transformations = {
        **{f'cat{i}': 'log1p' for i in range(1, 11)},
        **{f'xgb{i}': 'log1p' for i in range(1, 11)},
        **{f'cat{i}': 'yeo_johnson' for i in range(11, 16)},
        **{f'xgb{i}': 'yeo_johnson' for i in range(11, 16)}
    }
    
    print("Starting Out-of-Fold Prediction Generation...")
    print("=" * 60)
    
    for model_name, base_model in model_dict.items():
        print(f"\nProcessing {model_name} with {transformations.get(model_name, 'log1p')} transformation:")
        print("-" * 50)
        
        oof_preds = np.zeros(n_samples)
        fold_scores = []
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            print(f"  Fold {fold + 1}/{cv_folds}... ", end="")
            
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            transformation_type = transformations.get(model_name, 'log1p')
            
            if transformation_type == 'log1p':
                y_train_transformed = np.log1p(y_train)
                y_val_transformed = np.log1p(y_val)
            elif transformation_type == 'yeo_johnson':
                yj_transformer = PowerTransformer(method='yeo-johnson', standardize=False)
                y_train_transformed = yj_transformer.fit_transform(y_train.values.reshape(-1, 1)).ravel()
                y_val_transformed = yj_transformer.transform(y_val.values.reshape(-1, 1)).ravel()
            
            # UNIVERSAL MODEL CLONING
            model = clone(base_model)
            
            if model_name.startswith('cat'):
                # CatBoost with Pool
                train_pool = Pool(X_train, y_train_transformed, cat_features=categorical_features)
                val_pool = Pool(X_val, y_val_transformed, cat_features=categorical_features)
                
                model.set_params(verbose=False)
                model.fit(train_pool, eval_set=val_pool, verbose=False)
                val_preds_transformed = model.predict(val_pool)
                
            elif model_name.startswith('xgb'):
                # XGBoost with sklearn interface (arrays)
                model.set_params(verbose=False)
                model.fit(
                    X_train.values, 
                    y_train_transformed,
                    eval_set=[(X_val.values, y_val_transformed)],
                    verbose=False
                )
                val_preds_transformed = model.predict(X_val.values)
            
            # Inverse transformation
            if transformation_type == 'log1p':
                val_preds = np.expm1(val_preds_transformed)
            elif transformation_type == 'yeo_johnson':
                val_preds = yj_transformer.inverse_transform(val_preds_transformed.reshape(-1, 1)).ravel()
            
            val_preds = np.maximum(val_preds, 0)
            oof_preds[val_idx] = val_preds
            
            fold_score = np.sqrt(mean_squared_log_error(y_val, val_preds))
            fold_scores.append(fold_score)
            print(f"RMSLE: {fold_score:.6f}")
        
        oof_predictions[model_name] = oof_preds
        overall_score = np.sqrt(mean_squared_log_error(y, oof_preds))
        print(f"  Overall {model_name} RMSLE: {overall_score:.6f}")
        print(f"  Fold RMSLE: {np.mean(fold_scores):.6f} ± {np.std(fold_scores):.6f}")
    
    print("\n" + "=" * 60)
    print("OOF Prediction Generation Complete!")
    return oof_predictions


def generate_test_predictions(X_train, y_train, X_test, model_dict, categorical_features=None, save_dir="/kaggle/working/models"):
    test_predictions = pd.DataFrame(index=range(len(X_test)))
    transformations = {
        **{f'cat{i}': 'log1p' for i in range(1, 11)},
        **{f'xgb{i}': 'log1p' for i in range(1, 11)},
        **{f'cat{i}': 'yeo_johnson' for i in range(11, 16)},
        **{f'xgb{i}': 'yeo_johnson' for i in range(11, 16)}
    }
    
    print("Starting Test Prediction Generation...")
    print("=" * 60)
    
    os.makedirs(save_dir, exist_ok=True)
    
    for model_name, base_model in model_dict.items():
        transformation_type = transformations.get(model_name, 'log1p')
        print(f"\nTraining {model_name} with {transformation_type} transformation:")
        print("-" * 50)
        
        # Apply transformation
        if transformation_type == 'log1p':
            y_transformed = np.log1p(y_train)
            transformer = None
        elif transformation_type == 'yeo_johnson':
            yj_transformer = PowerTransformer(method='yeo-johnson', standardize=False)
            y_transformed = yj_transformer.fit_transform(y_train.values.reshape(-1, 1)).ravel()
            transformer = yj_transformer
        
        # UNIVERSAL MODEL CLONING
        model = clone(base_model)
        
        if model_name.startswith('cat'):
            # CatBoost with Pool
            train_pool = Pool(X_train, y_transformed, cat_features=categorical_features)
            test_pool = Pool(X_test, cat_features=categorical_features)
            
            model.set_params(verbose=False)
            model.fit(train_pool, verbose=False)
            
            # Save model
            model_path = os.path.join(save_dir, f"{model_name}.cbm")
            model.save_model(model_path)
            test_preds_transformed = model.predict(test_pool)
            
        elif model_name.startswith('xgb'):
            # XGBoost with sklearn interface (arrays)
            model.set_params(verbose=False)
            model.fit(X_train.values, y_transformed)
            
            # Save model
            model_path = os.path.join(save_dir, f"{model_name}.json")
            model.save_model(model_path)
            test_preds_transformed = model.predict(X_test.values)
        
        print(f"  Saved model to: {model_path}")
        
        # Inverse transformation
        if transformation_type == 'log1p':
            test_preds = np.expm1(test_preds_transformed)
        elif transformation_type == 'yeo_johnson':
            test_preds = transformer.inverse_transform(test_preds_transformed.reshape(-1, 1)).ravel()
        
        test_preds = np.maximum(test_preds, 0)
        test_predictions[model_name] = test_preds
        print(f"  Predictions generated - Mean: {test_preds.mean():.4f}")
    
    print("\n" + "=" * 60)
    print("Test Prediction Generation Complete!")
    return test_predictions

model_dict = {
    # Top 10 Performers (Best RMSLE scores)
    'cat1': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.013746, 'depth': 10, 'l2_leaf_reg': 2.698991,
        'random_strength': 2.10078, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2249,
        'border_count': 100, 'subsample': 0.92971, 'min_data_in_leaf': 69, 'random_state': seed+1
    }),
    'cat2': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.015832, 'depth': 11, 'l2_leaf_reg': 0.349602,
        'random_strength': 1.64188, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 1942,
        'border_count': 101, 'subsample': 0.93569, 'min_data_in_leaf': 83, 'random_state': seed+2
    }),
    'cat3': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.006420, 'depth': 13, 'l2_leaf_reg': 0.004398,
        'random_strength': 2.75674, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2964,
        'border_count': 74, 'subsample': 0.89261, 'min_data_in_leaf': 31, 'random_state': seed+3
    }),
    'cat4': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.013490, 'depth': 11, 'l2_leaf_reg': 0.000440,
        'random_strength': 1.66404, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2962,
        'border_count': 79, 'subsample': 0.89010, 'min_data_in_leaf': 44, 'random_state': seed+4
    }),
    'cat5': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.018602, 'depth': 13, 'l2_leaf_reg': 1.412277,
        'random_strength': 1.19449, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 1366,
        'border_count': 103, 'subsample': 0.95810, 'min_data_in_leaf': 94, 'random_state': seed+5
    }),
    'cat6': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.012350, 'depth': 12, 'l2_leaf_reg': 0.182027,
        'random_strength': 1.13882, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 1765,
        'border_count': 100, 'subsample': 0.87510, 'min_data_in_leaf': 79, 'random_state': seed+6
    }),
    'cat7': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.006722, 'depth': 12, 'l2_leaf_reg': 0.000298,
        'random_strength': 2.72937, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2832,
        'border_count': 94, 'subsample': 0.84665, 'min_data_in_leaf': 60, 'random_state': seed+7
    }),
    'cat8': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.008025, 'depth': 13, 'l2_leaf_reg': 0.002097,
        'random_strength': 2.76358, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2665,
        'border_count': 70, 'subsample': 0.86397, 'min_data_in_leaf': 41, 'random_state': seed+8
    }),
    'cat9': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.011477, 'depth': 13, 'l2_leaf_reg': 5.256055,
        'random_strength': 0.42451, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 1855,
        'border_count': 110, 'subsample': 0.90999, 'min_data_in_leaf': 30, 'random_state': seed+9
    }),
    'cat10': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.007692, 'depth': 12, 'l2_leaf_reg': 0.044905,
        'random_strength': 0.77312, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2947,
        'border_count': 89, 'subsample': 0.86828, 'min_data_in_leaf': 29, 'random_state': seed+10
    }),

    # Diverse Models (Different hyperparameter configurations)
    'cat11': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.149922, 'depth': 4, 'l2_leaf_reg': 0.000954,
        'random_strength': 0.13577, 'grow_policy': 'Depthwise',
        'bootstrap_type': 'Bernoulli', 'iterations': 1713,
        'border_count': 82, 'subsample': 0.96043, 'min_data_in_leaf': 26, 'random_state': seed+11
    }),
    'cat12': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.009778, 'depth': 5, 'l2_leaf_reg': 0.003320,
        'random_strength': 1.57431, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bayesian', 'iterations': 1584,
        'border_count': 87, 'min_data_in_leaf': 36, 'random_state': seed+12
    }),
    'cat13': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.019906, 'depth': 13, 'l2_leaf_reg': 0.457056,
        'random_strength': 1.79601, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2202,
        'border_count': 110, 'subsample': 0.96648, 'min_data_in_leaf': 37, 'random_state': seed+13
    }),
    'cat14': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.033327, 'depth': 9, 'l2_leaf_reg': 0.000170,
        'random_strength': 1.82267, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2617,
        'border_count': 83, 'subsample': 0.88803, 'min_data_in_leaf': 29, 'random_state': seed+14
    }),
    'cat15': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.031065, 'depth': 4, 'l2_leaf_reg': 3.520481,
        'random_strength': 0.77641, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 1369,
        'border_count': 127, 'subsample': 0.97896, 'min_data_in_leaf': 68, 'random_state': seed+15
    }),
    'cat16': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.036289, 'depth': 12, 'l2_leaf_reg': 3.208846,
        'random_strength': 1.81915, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2466,
        'border_count': 98, 'subsample': 0.80658, 'min_data_in_leaf': 46, 'random_state': seed+16
    }),
    'cat17': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.015127, 'depth': 12, 'l2_leaf_reg': 0.044913,
        'random_strength': 0.88674, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 1314,
        'border_count': 89, 'has_time': True, 'subsample': 0.86067,
        'min_data_in_leaf': 77, 'random_state': seed+17
    }),
    'cat18': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.007834, 'depth': 12, 'l2_leaf_reg': 0.000188,
        'random_strength': 1.19145, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2969,
        'border_count': 76, 'subsample': 0.88556, 'min_data_in_leaf': 42, 'random_state': seed+18
    }),
    'cat19': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.005942, 'depth': 13, 'l2_leaf_reg': 3.519287,
        'random_strength': 2.20966, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2734,
        'border_count': 67, 'subsample': 0.90607, 'min_data_in_leaf': 31, 'random_state': seed+19
    }),
    'cat20': CatBoostRegressor(**{
        'task_type': 'GPU',
        'learning_rate': 0.008074, 'depth': 12, 'l2_leaf_reg': 0.000154,
        'random_strength': 1.42529, 'grow_policy': 'SymmetricTree',
        'bootstrap_type': 'Bernoulli', 'iterations': 2940,
        'border_count': 76, 'subsample': 0.89114, 'min_data_in_leaf': 34, 'random_state': seed+20
    })
}

def create_model_dict(seed=seed):
    # Top 20 trials sorted by score (ascending)
    top_trials = [
        {'learning_rate': 0.005083, 'max_depth': 10, 'min_child_weight': 0.1395, 'subsample': 0.8310,
         'colsample_bytree': 0.8060, 'colsample_bylevel': 0.8634, 'colsample_bynode': 0.9274,
         'reg_alpha': 0.000462, 'reg_lambda': 0.0476, 'gamma': 0.007818, 'max_delta_step': 6.8812,
         'n_estimators': 2870, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 87,
         'scale_pos_weight': 2.6815, 'max_cat_to_onehot': 1},  # Trial 106 (best)

        {'learning_rate': 0.010725, 'max_depth': 9, 'min_child_weight': 0.1139, 'subsample': 0.5774,
         'colsample_bytree': 0.7203, 'colsample_bylevel': 0.7391, 'colsample_bynode': 0.9827,
         'reg_alpha': 3.37e-6, 'reg_lambda': 2.93e-5, 'gamma': 8.45e-6, 'max_delta_step': 9.7342,
         'n_estimators': 4548, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 212,
         'scale_pos_weight': 1.8312, 'max_cat_to_onehot': 5},  # Trial 121

        {'learning_rate': 0.012735, 'max_depth': 8, 'min_child_weight': 0.2303, 'subsample': 0.7543,
         'colsample_bytree': 0.8886, 'colsample_bylevel': 0.9938, 'colsample_bynode': 0.7659,
         'reg_alpha': 0.000156, 'reg_lambda': 0.21299, 'gamma': 0.01833, 'max_delta_step': 4.8201,
         'n_estimators': 2625, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 110,
         'scale_pos_weight': 0.1081, 'max_cat_to_onehot': 1},  # Trial 123

        {'learning_rate': 0.006451, 'max_depth': 10, 'min_child_weight': 0.19999, 'subsample': 0.8157,
         'colsample_bytree': 0.8856, 'colsample_bylevel': 0.9826, 'colsample_bynode': 0.9802,
         'reg_alpha': 0.000186, 'reg_lambda': 0.25328, 'gamma': 0.013416, 'max_delta_step': 6.9531,
         'n_estimators': 1871, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 164,
         'scale_pos_weight': 0.1679, 'max_cat_to_onehot': 2},  # Trial 102

        {'learning_rate': 0.008894, 'max_depth': 7, 'min_child_weight': 0.1165, 'subsample': 0.7397,
         'colsample_bytree': 0.9203, 'colsample_bylevel': 0.5501, 'colsample_bynode': 0.9911,
         'reg_alpha': 9.26e-6, 'reg_lambda': 0.001512, 'gamma': 0.008014, 'max_delta_step': 6.3884,
         'n_estimators': 4156, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 264,
         'scale_pos_weight': 2.5502, 'max_cat_to_onehot': 6},  # Trial 114

        {'learning_rate': 0.005829, 'max_depth': 8, 'min_child_weight': 0.2082, 'subsample': 0.7023,
         'colsample_bytree': 0.7377, 'colsample_bylevel': 0.8288, 'colsample_bynode': 0.8138,
         'reg_alpha': 0.02487, 'reg_lambda': 1.708e-6, 'gamma': 0.000311, 'max_delta_step': 6.3249,
         'n_estimators': 4626, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 186,
         'scale_pos_weight': 2.6929, 'max_cat_to_onehot': 3},  # Trial 58

        {'learning_rate': 0.014913, 'max_depth': 12, 'min_child_weight': 0.5018, 'subsample': 0.8225,
         'colsample_bytree': 0.7975, 'colsample_bylevel': 0.9770, 'colsample_bynode': 0.7928,
         'reg_alpha': 1.2734, 'reg_lambda': 0.0189, 'gamma': 0.001216, 'max_delta_step': 3.2869,
         'n_estimators': 2822, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 99,
         'scale_pos_weight': 2.9850, 'max_cat_to_onehot': 4},  # Trial 76

        {'learning_rate': 0.010725, 'max_depth': 9, 'min_child_weight': 0.7080, 'subsample': 0.8153,
         'colsample_bytree': 0.8771, 'colsample_bylevel': 0.9459, 'colsample_bynode': 0.8879,
         'reg_alpha': 2.4606, 'reg_lambda': 0.7910, 'gamma': 0.01128, 'max_delta_step': 0.7271,
         'n_estimators': 2696, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 272,
         'scale_pos_weight': 3.9623, 'max_cat_to_onehot': 3},  # Trial 82

        {'learning_rate': 0.007009, 'max_depth': 14, 'min_child_weight': 0.1542, 'subsample': 0.8970,
         'colsample_bytree': 0.9043, 'colsample_bylevel': 0.6083, 'colsample_bynode': 0.7258,
         'reg_alpha': 0.3723, 'reg_lambda': 1.7508, 'gamma': 1.729e-8, 'max_delta_step': 7.1540,
         'n_estimators': 3778, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 217,
         'scale_pos_weight': 2.8116, 'max_cat_to_onehot': 6},  # Trial 134

        {'learning_rate': 0.012865, 'max_depth': 7, 'min_child_weight': 0.1083, 'subsample': 0.5875,
         'colsample_bytree': 0.9857, 'colsample_bylevel': 0.8953, 'colsample_bynode': 0.9458,
         'reg_alpha': 1.2417, 'reg_lambda': 2.51e-6, 'gamma': 2.448e-7, 'max_delta_step': 6.8302,
         'n_estimators': 2780, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 97,
         'scale_pos_weight': 5.6313, 'max_cat_to_onehot': 2},  # Trial 108

        # Remaining 10 models for Yeo-Johnson and Box-Cox (next best trials)
        {'learning_rate': 0.009934, 'max_depth': 9, 'min_child_weight': 0.1139, 'subsample': 0.5774,
         'colsample_bytree': 0.7203, 'colsample_bylevel': 0.7391, 'colsample_bynode': 0.9827,
         'reg_alpha': 3.37e-6, 'reg_lambda': 2.93e-5, 'gamma': 8.45e-6, 'max_delta_step': 9.7342,
         'n_estimators': 4548, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 212,
         'scale_pos_weight': 1.8312, 'max_cat_to_onehot': 5},  # Trial 121 (duplicate)

        {'learning_rate': 0.011177, 'max_depth': 11, 'min_child_weight': 0.1664, 'subsample': 0.8599,
         'colsample_bytree': 0.9581, 'colsample_bylevel': 0.9853, 'colsample_bynode': 0.9858,
         'reg_alpha': 2.01e-5, 'reg_lambda': 1.0355, 'gamma': 0.00191, 'max_delta_step': 5.5203,
         'n_estimators': 1446, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 131,
         'scale_pos_weight': 0.8538, 'max_cat_to_onehot': 4},  # Trial 104

        {'learning_rate': 0.019332, 'max_depth': 13, 'min_child_weight': 0.1840, 'subsample': 0.5284,
         'colsample_bytree': 0.8355, 'colsample_bylevel': 0.9098, 'colsample_bynode': 0.9515,
         'reg_alpha': 0.6159, 'reg_lambda': 0.0002, 'gamma': 3.847e-8, 'max_delta_step': 9.7405,
         'n_estimators': 2048, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 178,
         'scale_pos_weight': 0.4373, 'max_cat_to_onehot': 9},  # Trial 92

        {'learning_rate': 0.014700, 'max_depth': 15, 'min_child_weight': 0.1905, 'subsample': 0.7315,
         'colsample_bytree': 0.7554, 'colsample_bylevel': 0.9092, 'colsample_bynode': 0.9266,
         'reg_alpha': 0.000625, 'reg_lambda': 5.7668, 'gamma': 0.00279, 'max_delta_step': 8.0438,
         'n_estimators': 2464, 'booster': 'gbtree', 'grow_policy': 'lossguide', 'max_leaves': 88,
         'max_bin': 79, 'scale_pos_weight': 7.8659, 'max_cat_to_onehot': 4},  # Trial 42

        {'learning_rate': 0.008983, 'max_depth': 10, 'min_child_weight': 0.1133, 'subsample': 0.4776,
         'colsample_bytree': 0.8145, 'colsample_bylevel': 0.8798, 'colsample_bynode': 0.7961,
         'reg_alpha': 2.29e-7, 'reg_lambda': 0.000352, 'gamma': 0.005435, 'max_delta_step': 8.4745,
         'n_estimators': 3965, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 418,
         'scale_pos_weight': 3.1297, 'max_cat_to_onehot': 8},  # Trial 60

        # Box-Cox models (assumed from next best trials)
        {'learning_rate': 0.005486, 'max_depth': 11, 'min_child_weight': 0.1493, 'subsample': 0.5960,
         'colsample_bytree': 0.8650, 'colsample_bylevel': 0.7675, 'colsample_bynode': 0.9718,
         'reg_alpha': 0.00596, 'reg_lambda': 8.4105, 'gamma': 0.01176, 'max_delta_step': 8.5700,
         'n_estimators': 2597, 'booster': 'gbtree', 'grow_policy': 'lossguide', 'max_leaves': 76,
         'max_bin': 153, 'scale_pos_weight': 3.1721, 'max_cat_to_onehot': 6},  # Trial 38

        {'learning_rate': 0.018357, 'max_depth': 11, 'min_child_weight': 0.6378, 'subsample': 0.8702,
         'colsample_bytree': 0.6121, 'colsample_bylevel': 0.8548, 'colsample_bynode': 0.9953,
         'reg_alpha': 1.249e-5, 'reg_lambda': 0.00208, 'gamma': 4.866e-5, 'max_delta_step': 7.7886,
         'n_estimators': 3118, 'booster': 'gbtree', 'grow_policy': 'lossguide', 'max_leaves': 70,
         'max_bin': 98, 'scale_pos_weight': 1.9579, 'max_cat_to_onehot': 5},  # Trial 33

        {'learning_rate': 0.021269, 'max_depth': 11, 'min_child_weight': 0.8516, 'subsample': 0.4435,
         'colsample_bytree': 0.9108, 'colsample_bylevel': 0.7043, 'colsample_bynode': 0.9841,
         'reg_alpha': 0.000257, 'reg_lambda': 2.2067, 'gamma': 0.000195, 'max_delta_step': 7.0162,
         'n_estimators': 1841, 'booster': 'gbtree', 'grow_policy': 'lossguide', 'max_leaves': 66,
         'max_bin': 205, 'scale_pos_weight': 4.6618, 'max_cat_to_onehot': 1},  # Trial 36

        {'learning_rate': 0.029657, 'max_depth': 7, 'min_child_weight': 0.1266, 'subsample': 0.5557,
         'colsample_bytree': 0.7108, 'colsample_bylevel': 0.9464, 'colsample_bynode': 0.7707,
         'reg_alpha': 3.34e-6, 'reg_lambda': 0.02676, 'gamma': 1.256e-6, 'max_delta_step': 8.3364,
         'n_estimators': 1408, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 116,
         'scale_pos_weight': 2.9827, 'max_cat_to_onehot': 1},  # Trial 30

        {'learning_rate': 0.010821, 'max_depth': 7, 'min_child_weight': 1.0747, 'subsample': 0.9333,
         'colsample_bytree': 0.7553, 'colsample_bylevel': 0.5955, 'colsample_bynode': 0.8278,
         'reg_alpha': 5.10e-6, 'reg_lambda': 0.00452, 'gamma': 1.484e-6, 'max_delta_step': 4.6020,
         'n_estimators': 4936, 'booster': 'gbtree', 'grow_policy': 'lossguide', 'max_leaves': 116,
         'max_bin': 221, 'scale_pos_weight': 4.6267, 'max_cat_to_onehot': 5}  # Trial 16
    ]

    model_dict = {}
    for i in range(20):
        params = top_trials[i].copy()
        params['random_state'] = seed + i
        params['tree_method'] = 'gpu_hist'
        model_name = f'xgb{i+1}'
        model_dict[model_name] = XGBRegressor(**params)
    
    return model_dict
xgb_dict = create_model_dict()

In [11]:
X = X_new.drop(columns=['id', 'bmi_hr'])

In [12]:
xgb_oof_preds = generate_oof_predictions(
        X=X, 
        y=y, 
        model_dict=xgb_dict,
        cv_folds=10,
        categorical_features=None
    )

Starting Out-of-Fold Prediction Generation...

Processing xgb1 with log1p transformation:
--------------------------------------------------
  Fold 1/10... RMSLE: 0.060168
  Fold 2/10... RMSLE: 0.058601
  Fold 3/10... RMSLE: 0.061183
  Fold 4/10... RMSLE: 0.058832
  Fold 5/10... RMSLE: 0.058925
  Fold 6/10... RMSLE: 0.060135
  Fold 7/10... RMSLE: 0.058774
  Fold 8/10... RMSLE: 0.060595
  Fold 9/10... RMSLE: 0.059483
  Fold 10/10... RMSLE: 0.058339
  Overall xgb1 RMSLE: 0.059511
  Fold RMSLE: 0.059504 ± 0.000913

Processing xgb2 with log1p transformation:
--------------------------------------------------
  Fold 1/10... RMSLE: 0.060568
  Fold 2/10... RMSLE: 0.059353
  Fold 3/10... RMSLE: 0.061883
  Fold 4/10... RMSLE: 0.059152
  Fold 5/10... RMSLE: 0.059599
  Fold 6/10... RMSLE: 0.060506
  Fold 7/10... RMSLE: 0.059332
  Fold 8/10... RMSLE: 0.061196
  Fold 9/10... RMSLE: 0.060153
  Fold 10/10... RMSLE: 0.058979
  Overall xgb2 RMSLE: 0.060079
  Fold RMSLE: 0.060072 ± 0.000912

Processing 

In [13]:
# oof_preds = generate_oof_predictions(
#         X=X, 
#         y=y, 
#         model_dict=model_dict,
#         cv_folds=10,
#         categorical_features=None
#     )

In [14]:
xgb_test_preds =  generate_test_predictions(X_train=X, y_train=y, model_dict=xgb_dict, X_test=test.drop(columns='id'))

Starting Test Prediction Generation...

Training xgb1 with log1p transformation:
--------------------------------------------------
  Saved model to: /kaggle/working/models/xgb1.json
  Predictions generated - Mean: 88.1663

Training xgb2 with log1p transformation:
--------------------------------------------------
  Saved model to: /kaggle/working/models/xgb2.json
  Predictions generated - Mean: 88.1759

Training xgb3 with log1p transformation:
--------------------------------------------------
  Saved model to: /kaggle/working/models/xgb3.json
  Predictions generated - Mean: 88.1710

Training xgb4 with log1p transformation:
--------------------------------------------------
  Saved model to: /kaggle/working/models/xgb4.json
  Predictions generated - Mean: 88.1664

Training xgb5 with log1p transformation:
--------------------------------------------------
  Saved model to: /kaggle/working/models/xgb5.json
  Predictions generated - Mean: 88.1739

Training xgb6 with log1p transformation:

In [15]:
# test_preds.head(3)

In [16]:
# def analyze_oof_predictions(oof_predictions, y_true):
   
#     results = {}
    
#     print("OOF Predictions Analysis:")
#     print("=" * 40)
    
#     for col in oof_predictions.columns:
#         try:
#             score = np.sqrt(mean_squared_log_error(y_true, oof_predictions[col]))
#             results[col] = {
#                 'RMSLE': score,
#                 'Mean_Pred': oof_predictions[col].mean(),
#                 'Std_Pred': oof_predictions[col].std(),
#                 'Min_Pred': oof_predictions[col].min(),
#                 'Max_Pred': oof_predictions[col].max()
#             }
#             print(f"{col}: RMSLE = {score:.6f}")
#         except ValueError as e:
#             print(f"{col}: Error calculating RMSLE - {e}")
#             results[col] = {
#                 'RMSLE': np.nan,
#                 'Mean_Pred': oof_predictions[col].mean(),
#                 'Std_Pred': oof_predictions[col].std(),
#                 'Min_Pred': oof_predictions[col].min(),
#                 'Max_Pred': oof_predictions[col].max()
#             }
    
#     # Create correlation matrix
#     print(f"\nCorrelation matrix shape: {oof_predictions.corr().shape}")
#     print(f"Average correlation between models: {oof_predictions.corr().values[np.triu_indices_from(oof_predictions.corr().values, k=1)].mean():.4f}")
    
#     return pd.DataFrame(results).T


In [17]:
xgb_oof_preds.to_csv('xgb_oof_preds.csv', index=False)
xgb_test_preds.to_csv('xgb_test_preds.csv', index=False)

In [18]:
# results = analyze_oof_predictions(oof_preds, y)

In [19]:
# results

In [20]:
# scores = cv_score(X_new.drop(columns=['id', 'bmi_hr']), y, model_dict)

In [21]:
# scores

In [22]:
# scores.T.to_csv('model_cv_scores.csv', index=False)

In [23]:
# scores_df = pd.DataFrame(scores)
# scores_df.T.to_csv('models_cv_scores.csv', index=False)

In [24]:
# cb = cat.CatBoostRegressor(random_state=0, verbose=0, **params)
# # etr = ExtraTreesRegressor(random_state=seed, n_jobs=-1)
# y_log1p = np.log1p(y)
# af = AutoFeatRegressor(feateng_steps=3, verbose=1, n_jobs=1)
# X_train_new = af.fit_transform(X, y_log1p)
# X_test_new = af.transform(test)


In [25]:
# X_train_new.to_csv('X_train_new_3step.csv', index=False)
# X_test_new.to_csv('X_test_new_3step.csv')

In [26]:
# cb.fit(X_train_new, y_log1p)

In [27]:
# # # from scipy.special import inv_boxcox
# # # # preds_bc = model.predict(X_test_new)
# # # # preds = inv_boxcox(preds_bc, lambda_)
# preds_log1p = cb.predict(X_test_new)
# preds = np.expm1(preds_log1p)
# # preds = np.clip(preds, 1, None)
# samp_sub['Calories'] = preds
# samp_sub.to_csv('submission.csv', index=False)

In [28]:

# from sklearn.feature_selection import RFECV
# from sklearn.metrics import make_scorer
# from sklearn.model_selection import KFold, check_cv
# from sklearn.base import BaseEstimator, clone
# from sklearn.utils.metaestimators import if_delegate_has_method
# from sklearn.utils import check_X_y
# import numpy as np
# import pandas as pd
# import catboost as cat
# from sklearn.metrics import mean_squared_log_error
# from joblib import Parallel, delayed

# def rmsle(y_true, y_pred):
#     """Root Mean Squared Logarithmic Error"""
#     # Ensure predictions are positive
#     y_pred = np.clip(y_pred, 1e-6, None)
#     return np.sqrt(mean_squared_log_error(y_true, y_pred))

# class PoolRFECV(RFECV):
#     """
#     Recursive Feature Elimination with Cross-Validation specifically designed for CatBoost
#     that uses Pool objects for efficient GPU memory utilization.
#     """
    
#     def __init__(self, estimator, step=1, min_features_to_select=1, cv=5, 
#                  scoring=None, verbose=0, n_jobs=None, importance_getter='auto'):
        
#         super().__init__(
#             estimator=estimator,
#             step=step,
#             min_features_to_select=min_features_to_select,
#             cv=cv,
#             scoring=scoring,
#             verbose=verbose,
#             n_jobs=n_jobs,
#             importance_getter=importance_getter
#         )
    
#     def _fit(self, X, y, step_score=None):
#         """
#         Custom fit implementation that uses CatBoost Pool objects
#         """
#         # Initial setup similar to parent class
#         X, y = check_X_y(X, y, ensure_min_features=2, force_all_finite='allow-nan')
        
#         # Store the original feature names
#         if isinstance(X, pd.DataFrame):
#             self.feature_names = X.columns.tolist()
#         else:
#             self.feature_names = [f"feature_{i}" for i in range(X.shape[1])]
            
#         # Setup cross-validation strategy
#         cv = check_cv(self.cv, y)
        
#         # Create the scoring function
#         scorer = self.scoring
#         if callable(self.scoring):
#             scorer = self.scoring
#         elif self.scoring is None or isinstance(self.scoring, str):
#             scorer = make_scorer(self.scoring)

#         # Initialize variables
#         n_features = X.shape[1]
#         if 0.0 < self.step < 1.0:
#             step = int(max(1, self.step * n_features))
#         else:
#             step = int(self.step)
        
#         # Create the actual step based on n_features
#         if step <= 0:
#             raise ValueError("Step must be >0")
            
#         # Define the number of features to be eliminated in each iteration
#         # and determine all possible feature counts
#         if step == 1:
#             n_feature_subsets = n_features - self.min_features_to_select + 1
#         else:
#             n_feature_subsets = (n_features - self.min_features_to_select) // step + 1
#             if (n_features - self.min_features_to_select) % step > 0:
#                 n_feature_subsets += 1
                
#         # Pre-allocate scores array and support mask
#         scores = np.zeros(n_feature_subsets)
#         self.cv_results_ = {}
        
#         # Start with all features
#         support_ = np.ones(n_features, dtype=bool)
#         ranking_ = np.ones(n_features, dtype=int)
        
#         # Results containers
#         self.cv_results_["mean_test_score"] = np.zeros(n_feature_subsets)
#         self.cv_results_["std_test_score"] = np.zeros(n_feature_subsets)
        
#         # Define a function to evaluate a subset of features
#         def _evaluate_subset(train_indices, test_indices, features):
#             # Get training and testing data using the specified indices and feature subset
#             X_train, X_test = X[train_indices][:, features], X[test_indices][:, features]
#             y_train, y_test = y[train_indices], y[test_indices]
            
#             # Create Pool objects for efficient GPU memory usage
#             train_pool = cat.Pool(
#                 data=X_train,
#                 label=y_train,
#                 feature_names=[self.feature_names[i] for i in np.where(features)[0]]
#             )
            
#             test_pool = cat.Pool(
#                 data=X_test,
#                 label=y_test,
#                 feature_names=[self.feature_names[i] for i in np.where(features)[0]]
#             )
            
#             # Clone the estimator to ensure clean fitting
#             est = clone(self.estimator)
            
#             # Fit on train pool
#             est.fit(train_pool, verbose=False)
            
#             # Use fitted estimator to predict test data
#             y_pred = est.predict(test_pool)
            
#             # Return the score
#             return scorer._score_func(y_test, y_pred)
        
#         # Feature subset evaluation
#         for feature_subset_idx, feature_subset_size in enumerate(range(n_features, self.min_features_to_select - 1, -step)):
#             # Determine features for this subset
#             if feature_subset_idx == 0:
#                 current_features = np.ones(n_features, dtype=bool)
#             else:
#                 # Get important features from the current estimator
#                 estimator = clone(self.estimator)
                
#                 # Create a Pool with current features
#                 pool = cat.Pool(
#                     data=X[:, current_features],
#                     label=y,
#                     feature_names=[self.feature_names[i] for i in np.where(current_features)[0]]
#                 )
                
#                 # Fit the model
#                 estimator.fit(pool, verbose=False)
                
#                 # Get feature importances
#                 if hasattr(estimator, 'feature_importances_'):
#                     importances = estimator.feature_importances_
#                 elif hasattr(estimator, 'get_feature_importance'):
#                     importances = estimator.get_feature_importance()
#                 else:
#                     raise ValueError("Estimator does not have feature_importances_ or get_feature_importance method")
                
#                 # Eliminate the least important features
#                 # First, we need to ensure indices are relative to the active features
#                 active_indices = np.where(current_features)[0]
                
#                 # Sort by importance but use indices of active features
#                 importance_order = np.argsort(importances)
                
#                 # Calculate number of features to eliminate in this step
#                 to_eliminate = min(step, np.sum(current_features) - self.min_features_to_select)
                
#                 # Update support mask to remove the least important features
#                 # Only eliminate up to the number of available features
#                 to_eliminate = min(to_eliminate, len(importance_order))
                
#                 # Get the original indices of features to eliminate
#                 for i in range(to_eliminate):
#                     if i < len(importance_order):
#                         feature_idx = importance_order[i]
#                         if feature_idx < len(active_indices):
#                             orig_idx = active_indices[feature_idx]
#                             current_features[orig_idx] = False
            
#             # Now evaluate this feature subset using cross-validation
#             cv_scores = []
            
#             # Get split indices
#             split_indices = list(cv.split(X, y))
            
#             # Evaluate the current feature subset using parallel processing if required
#             if self.n_jobs and self.n_jobs != 1:
#                 cv_scores = Parallel(n_jobs=self.n_jobs)(
#                     delayed(_evaluate_subset)(
#                         train_indices, test_indices, current_features
#                     )
#                     for train_indices, test_indices in split_indices
#                 )
#             else:
#                 for train_indices, test_indices in split_indices:
#                     cv_scores.append(_evaluate_subset(train_indices, test_indices, current_features))
            
#             # Calculate mean and std of scores
#             mean_score = np.mean(cv_scores)
#             std_score = np.std(cv_scores)
            
#             # Store the results
#             self.cv_results_["mean_test_score"][feature_subset_idx] = mean_score
#             self.cv_results_["std_test_score"][feature_subset_idx] = std_score
            
#             # If this is the first subset, no need to compare with previous
#             if feature_subset_idx == 0:
#                 best_subset_size = feature_subset_size
#                 best_subset_idx = feature_subset_idx
#             # Otherwise, check if current subset is better than the best so far
#             elif mean_score > self.cv_results_["mean_test_score"][best_subset_idx]:
#                 best_subset_size = feature_subset_size
#                 best_subset_idx = feature_subset_idx
            
#             # Print progress if verbose
#             if self.verbose > 0:
#                 print(f"Features: {np.sum(current_features)}/{n_features} - Score: {mean_score:.4f} ± {std_score:.4f}")
            
#             # Update support and ranking
#             ranking_[~current_features] = n_features - np.sum(current_features) + 1
            
#             # Check if we've reached the minimum features to select
#             if np.sum(current_features) <= self.min_features_to_select:
#                 break
        
#         # Find the subset with the best score
#         best_idx = np.argmax(self.cv_results_["mean_test_score"])
        
#         # If the best score is not from the first subset, go back and determine which features to keep
#         if best_idx > 0:
#             # Start with all features
#             temp_support = np.ones(n_features, dtype=bool)
            
#             # Recreate the elimination process to determine the best subset
#             for feature_subset_idx in range(best_idx):
#                 # Create a Pool with current features
#                 estimator = clone(self.estimator)
                
#                 # Get indices where features are active
#                 active_indices = np.where(temp_support)[0]
                
#                 pool = cat.Pool(
#                     data=X[:, temp_support],
#                     label=y,
#                     feature_names=[self.feature_names[i] for i in active_indices]
#                 )
                
#                 # Fit the model
#                 estimator.fit(pool, verbose=False)
                
#                 # Get feature importances
#                 if hasattr(estimator, 'feature_importances_'):
#                     importances = estimator.feature_importances_
#                 elif hasattr(estimator, 'get_feature_importance'):
#                     importances = estimator.get_feature_importance()
#                 else:
#                     raise ValueError("Estimator does not have feature_importances_ or get_feature_importance method")
                
#                 # Sort features by importance (ascending)
#                 importance_order = np.argsort(importances)
                
#                 # Calculate number of features to eliminate in this step
#                 to_eliminate = min(step, np.sum(temp_support) - self.min_features_to_select)
                
#                 # Only eliminate up to the number of available features
#                 to_eliminate = min(to_eliminate, len(importance_order))
                
#                 # Update support mask to remove the least important features
#                 for i in range(to_eliminate):
#                     if i < len(importance_order):
#                         feature_idx = importance_order[i]
#                         if feature_idx < len(active_indices):
#                             orig_idx = active_indices[feature_idx]
#                             temp_support[orig_idx] = False
            
#             # Set the final support and ranking
#             self.support_ = temp_support
            
#         else:
#             # If the best is the first subset with all features, support is already all ones
#             self.support_ = np.ones(n_features, dtype=bool)
        
#         # Update ranking based on final support
#         self.ranking_ = np.ones(n_features, dtype=int)
#         self.ranking_[~self.support_] = 2  # 1 for selected, 2 for not selected (simple ranking)
        
#         # Store the optimal number of features
#         self.n_features_ = np.sum(self.support_)
        
#         # Final fit with selected features
#         self.estimator_ = clone(self.estimator)
        
#         pool = cat.Pool(
#             data=X[:, self.support_],
#             label=y,
#             feature_names=[self.feature_names[i] for i in np.where(self.support_)[0]]
#         )
        
#         self.estimator_.fit(pool, verbose=False)
        
#         return self

#     def fit(self, X, y):
#         """
#         Fit the RFE model with CatBoost Pool optimization.
        
#         Parameters
#         ----------
#         X : array-like of shape (n_samples, n_features)
#             Training data.
#         y : array-like of shape (n_samples,)
#             Target values.
        
#         Returns
#         -------
#         self : object
#             Returns the instance itself.
#         """
#         return self._fit(X, y)
    
#     @if_delegate_has_method(['estimator', 'estimator_'])
#     def predict(self, X):
#         """Reduce X to the selected features and predict using the estimator."""
#         # Convert data to numpy if not already
#         X_numpy = X.values if isinstance(X, pd.DataFrame) else X
        
#         # Create Pool for prediction
#         pool = cat.Pool(
#             data=X_numpy[:, self.support_],
#             feature_names=[self.feature_names[i] for i in np.where(self.support_)[0]]
#         )
        
#         return self.estimator_.predict(pool)

# def rfecv_feature_selector(X_transformed, y_target, model, n_splits=5, seed=42):
#     """
#     RFECV feature selection with CatBoost Pool integration for GPU memory optimization
    
#     Parameters
#     ----------
#     X_transformed : pandas.DataFrame
#         The feature matrix
#     y_target : pandas.Series
#         The target variable
#     model : CatBoostRegressor
#         The CatBoost model to use for feature selection
#     n_splits : int, default=5
#         Number of CV folds
#     seed : int, default=42
#         Random seed for reproducibility
    
#     Returns
#     -------
#     tuple : (best_score, best_features)
#         The best CV score and the list of selected features
#     """
#     # Define a custom scoring function for RMSLE - fixed to handle log-transformed targets
#     def _custom_rmsle(y_true_log, y_pred_log):
#         """
#         Calculate RMSLE after converting predictions from log space back to original space
        
#         Parameters:
#         -----------
#         y_true_log : array-like
#             Log-transformed true target values used during training
#         y_pred_log : array-like
#             Log-transformed predictions from the model
            
#         Returns:
#         --------
#         float : Negative RMSLE score (negative because sklearn maximizes scores)
#         """
#         # Convert both true values and predictions back from log space
#         y_true_orig = np.expm1(y_true_log)
#         y_pred_orig = np.expm1(y_pred_log)
        
#         # Apply min clip to predictions to avoid negative values
#         y_pred_orig = np.clip(y_pred_orig, 1e-6, None)
        
#         # Calculate RMSLE in original space
#         score = rmsle(y_true_orig, y_pred_orig)
        
#         # CHANGE: Store the raw RMSLE score without negation
#         # We'll handle the negation separately for sklearn
#         return score
    
#     # CHANGE: Create scorer with greater_is_better=False to avoid negation in the scoring function
#     custom_scorer = make_scorer(_custom_rmsle, greater_is_better=False)
    
#     # 1. Log transform target for CatBoost
#     y_log = np.log1p(y_target)
    
#     # 2. Configure RFECV with our custom Pool-optimized implementation
#     selector = PoolRFECV(
#         estimator=model,
#         step=1,  # Remove multiple features at each step
#         cv=KFold(n_splits, shuffle=True, random_state=seed),
#         scoring=custom_scorer,
#         min_features_to_select=16,
#         n_jobs=1,  # Set to 1 for GPU memory optimization
#         verbose=1
#     )
    
#     # 3. Convert DataFrame to NumPy arrays for our custom RFECV implementation
#     X_numpy = X_transformed.values if isinstance(X_transformed, pd.DataFrame) else X_transformed
    
#     # 4. Feature elimination with Pool optimization
#     selector.fit(X_numpy, y_log)
    
#     # 5. Get selected features
#     feature_names = X_transformed.columns.tolist()
#     best_features = [feature_names[i] for i in range(len(feature_names)) if selector.support_[i]]
    
#     # 6. CHANGED: Get the best score directly since we're using greater_is_better=False
#     # This means the minimum score in cv_results is the best one
#     if len(selector.cv_results_['mean_test_score']) > 0:
#         # Now we get the minimum since lower RMSLE is better and greater_is_better=False
#         best_score = np.min(selector.cv_results_['mean_test_score'])
        
#         # Debug lines (remove in production)
#         print("Raw CV results scores:", selector.cv_results_['mean_test_score'])
#         print("Best index:", np.argmin(selector.cv_results_['mean_test_score']))
#         print("Min raw score (should be positive RMSLE):", best_score)
        
#         # Ensure score is never shown as exactly 0.0 if it's a small but real value
#         if 0 < best_score < 0.00001:
#             best_score = max(0.00001, best_score)
#     else:
#         best_score = float('inf')  # Default if no scores available
    
#     return best_score, best_features

In [29]:
# best_score, selected_features = rfecv_feature_selector(
#     X_transformed=X_train_new,
#     y_target=y,
#     model=model,
#     n_splits=5,
#     seed=42
# )


In [30]:
# best_score

In [31]:
# selected_features